In [1]:
!python --version # Python 3.11.11

Python 3.11.11


In [4]:
!pip install -qU transformers sentence-transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 97.1 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.2 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

In [5]:
# faiss-cpu with Python 3.11.11 is still fast
# In this case, you do not need to use faiss-gpu
# Enable P100 GPU only to accelerate embedding model

!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 52.5 MB/s eta 0:00:00:00:0100:01


In [130]:
"""
Name: transformers
Version: 4.51.3
Name: sentence-transformers
Version: 4.1.0
Name: datasets
Version: 3.6.0
Name: faiss-cpu
Version: 1.11.0
"""

# for verbose info, keep only: !pip ... datasets
!pip show transformers sentence-transformers datasets faiss-cpu | grep -E '^Name:|^Version:'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: transformers
Version: 4.52.4
Name: sentence-transformers
Version: 4.1.0
Name: datasets
Version: 3.6.0
Name: faiss-cpu
Version: 1.11.0


In [6]:
from datasets import Dataset, DatasetDict, load_dataset

from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.cross_encoder import CrossEncoder, CrossEncoderTrainer, CrossEncoderTrainingArguments, losses

from sentence_transformers import SentenceTransformer
from transformers import EarlyStoppingCallback

import pandas as pd
import numpy as np
import torch
import random
import re

from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

2025-06-01 07:07:12.124780: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748761632.311391      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748761632.364513      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
user_secrets = UserSecretsClient()
secret_token = user_secrets.get_secret("UIT_21520296_DATASET")
login(token=secret_token)

In [8]:
def preprocess(element):
    # Replace newlines, tabs, and redundant spaces with a single space
    processed_element = re.sub(r'[\n\t\s]+', ' ', element)
    
    # Strip leading and trailing spaces
    processed_element = processed_element.strip()
    
    return processed_element

In [9]:
path_train = '/kaggle/input/uit-r2gqa/data-raw/train.csv'
path_valid = '/kaggle/input/uit-r2gqa/data-raw/val.csv'
path_test  = '/kaggle/input/uit-r2gqa/data-raw/test.csv'

df_train = pd.read_csv(path_train)
df_valid = pd.read_csv(path_valid)
df_test  = pd.read_csv(path_test)

In [10]:
print('train', df_train.shape, df_train['context'].unique().shape, df_train['question'].unique().shape)
print('valid', df_valid.shape, df_valid['context'].unique().shape, df_valid['question'].unique().shape)
print('test', df_test.shape, df_test['context'].unique().shape, df_test['question'].unique().shape)

train (7806, 8) (294,) (7710,)
valid (976, 8) (258,) (971,)
test (976, 8) (256,) (976,)


In [11]:
df_train.columns

Index(['index', 'context', 'article', 'document', 'question',
       'extractive answer', 'abstractive answer', 'yes/no'],
      dtype='object')

In [12]:
columns_to_drop = ['index', 'article', 'document', 'extractive answer', 'abstractive answer', 'yes/no']

In [13]:
df_train.drop(columns=columns_to_drop, inplace=True)
df_train = df_train.map(func=preprocess)

print(df_train.shape)
df_train.head(5)

(7806, 2)


,context,question
0,Điều 9. Tuyển bổ sung và loại ra khỏi chương t...,Sinh viên dự bị không trở thành sinh viên chín...
1,Điều 4. Kiểm tra xếp lớp đầu khóa cho sinh viê...,Các mức điểm xếp lớp tiếng Anh được Hiệu trưởn...
2,Điều 5. Chương trình đào tạo CT CLC được xây d...,Trình độ tiếng Nhật đạt N mấy mới thì sinh viê...
3,"Điều 25. Cách tính điểm trung bình 1. ĐTBHK, Đ...","Cách tính điểm giữa điểm trung bình học kỳ, đi..."
4,Điều 19. Tổ chức phúc khảo và giải quyết khiếu...,Hoạt động nào sẽ diễn ra khi SV có khiếu nại v...


In [14]:
df_valid.drop(columns=columns_to_drop, inplace=True)
df_valid = df_valid.map(func=preprocess)

print(df_valid.shape)
df_valid.head(5)

(976, 2)


,context,question
0,Điều 16. Xử lý học vụ Xử lý học vụ nhằm giúp c...,Sinh viên chưa hết thời gian tối đa hoàn thành...
1,"Điều 32. Chấm thực tập, khóa luận tốt nghiệp 1...",Điểm của khoá luận tính theo thang điểm mấy?
2,"Điều 11. Đào tạo ngoại ngữ - Vào đầu khóa học,...",Vào đầu mỗi khóa học sinh viên CT CLC cần phải...
3,"Điều 9. Văn bằng, chứng chỉ/chứng nhận được sử...",Tại sao sinh viên cần nộp chứng chỉ cho Trường?
4,Điều 1. Phạm vi điều chỉnh và đối tượng áp dụn...,"Văn bằng giáo dục đại học, sau đại học và các ..."


In [15]:
df_test.drop(columns=columns_to_drop, inplace=True)
df_test = df_test.map(func=preprocess)

print(df_test.shape)
df_test.head(5)

(976, 2)


,context,question
0,"Điều 8. Xây dựng, thẩm định học liệu điện tử 1...",Học liệu điện tử sau khi được thông qua bởi ĐV...
1,"Điều 20. Hồ sơ, trình tự, thủ tục chỉnh sửa nộ...",Có thể nộp bản sao giấy khai sinh cho hồ sơ đề...
2,Điều 23. Điểm Miễn 1. Điểm BL - Sinh viên đã t...,Quy định về điểm M trong trường hợp sinh viên ...
3,Điều 4. Xét duyệt đề tài KLTN Khoa chịu trách ...,Yêu cầu đối với học vị của CBHD hướng dẫn KLTN...
4,Điều 11. Quy trình chuẩn bị cho kỳ thi 1. Đầu ...,Công việc phân công cán bộ coi thi có nội dung...


In [16]:
hf_train = Dataset.from_pandas(df_train)
hf_train

Dataset({
    features: ['context', 'question'],
    num_rows: 7806
})

In [17]:
hf_valid = Dataset.from_pandas(df_valid)
hf_valid

Dataset({
    features: ['context', 'question'],
    num_rows: 976
})

In [18]:
hf_test = Dataset.from_pandas(df_test)
hf_test

Dataset({
    features: ['context', 'question'],
    num_rows: 976
})

Another way to use [**mine_hard_negatives**](https://sbert.net/docs/package_reference/util.html#sentence_transformers.util.mine_hard_negatives):

`output_format` can be set to `labeled-pair` which can be used with [BinaryCrossEntropyLoss](https://sbert.net/docs/package_reference/cross_encoder/losses.html#sentence_transformers.cross_encoder.losses.BinaryCrossEntropyLoss). <br>
This combination is easier to implement than combination of `output_format="triplet"` and [MultipleNegativesRankingLoss](https://sbert.net/docs/package_reference/cross_encoder/losses.html#sentence_transformers.cross_encoder.losses.MultipleNegativesRankingLoss)  

*(read section Recommendations in document of both losses for more info)*


In [19]:
from sentence_transformers.util import mine_hard_negatives

# **RE-USE** bi-encoder 'KhoaUIT/Halong-UIT-R2GQA' which is already trained on the same dataset
# Mine hard negatives using a very efficient embedding model
embedding_model = SentenceTransformer("KhoaUIT/Halong-UIT-R2GQA", device="cuda")


def make_hard_dataset(input_dataset, output_format="triplet", num_negatives=3):    
    # Params for input dataset:
    # anchor_column_name   (str, optional) – The column name in dataset that contains the anchor/query. Defaults to None, in which case the first column in dataset will be used.
    # positive_column_name (str, optional) – The column name in dataset that contains the positive candidates. Defaults to None, in which case the second column in dataset will be used.
    
    dataset_hard = mine_hard_negatives(
        input_dataset,
        embedding_model,
        anchor_column_name="question",
        positive_column_name="context",
        num_negatives=num_negatives,   # How many negatives per question-answer pair
        sampling_strategy="top",       # Sample the top negatives from the range
        range_min=8,                   # Skip the x most similar samples
        range_max=200,                 # Consider only the x most similar samples
        max_score=0.8,                 # Only consider samples with a similarity score of at most x
        absolute_margin=0.1,           # Anchor-negative similarity is at least x lower than anchor-positive similarity
        relative_margin=0.1,           # Anchor-negative similarity is at most 1-x times the anchor-positive similarity, e.g. 90%
        batch_size=512,                # Use a batch size for the embedding model
        output_format=output_format,   # ”triplet”: (anchor, positive, negative) triplets, i.e. 3 columns. Useful for e.g. CachedMultipleNegativesRankingLoss
        use_faiss=True,                # Using FAISS is recommended to keep memory usage low (pip install faiss-gpu or pip install faiss-cpu)
    )

    return dataset_hard

modules.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.84k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/56.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/305 [00:00<?, ?B/s]

In [20]:
train_hard = make_hard_dataset(hf_train, num_negatives=1)
train_hard, train_hard[0]

Found 7701 unique queries out of 7806 total queries.
Found an average of 1.014 positives per query.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Querying FAISS index: 100%|██████████| 1/1 [00:00<00:00,  5.41it/s]


Metric       Positive       Negative     Difference
Count           7,806          7,806               
Mean           0.8394         0.5545         0.2849
Median         0.8444         0.5532         0.2832
Std            0.0540         0.0360         0.0577
Min            0.5740         0.4305         0.1155
25%            0.8042         0.5289         0.2432
50%            0.8444         0.5532         0.2832
75%            0.8793         0.5785         0.3260
Max            0.9639         0.7187         0.4754
Skipped 7,282 potential negatives (0.47%) due to the absolute_margin of 0.1.
Skipped 112 potential negatives (0.01%) due to the max_score of 0.8.


(Dataset({
     features: ['question', 'context', 'negative'],
     num_rows: 7806
 }),
 {'question': 'Yêu cầu của giáo trình được sử dụng cho việc giảng dạy là gì?',
  'context': 'Điều 5. Giáo trình cho mỗi học phần 5.1. Mỗi học phần dùng một giáo trình chính. Trong từng giai đoạn, giáo trình cho học phần có thể thay đổi do Hiệu trưởng ra quyết định. 5.2. Ngoài một giáo trình chính, mỗi học phần được trường tổ chức biên soạn tối đa hai sách chuyên khảo, ba tài liệu tham khảo, một tài liệu hướng dẫn. 5.3. Các giáo trình sử dụng trong giảng dạy phải được ghi rõ trong đề cương học phần đã được Hiệu trưởng phê duyệt.',
  'negative': 'Điều 13. Kinh phí giáo trình ĐHQG hỗ trợ cho xuất bản giáo trình 13.1. Đối với giáo trình xuất bản Để thúc đẩy cho công tác giáo trình phát triển nhanh, đáp ứng kịp cho nhu cầu đào tạo, ĐHQG chi hỗ trợ một phần kinh phí. Cụ thể là: a) Hỗ trợ 50% cho phí in đối với giáo trình

In [21]:
valid_hard = make_hard_dataset(hf_valid, num_negatives=1)
valid_hard, valid_hard[0]

Found 971 unique queries out of 976 total queries.
Found an average of 1.005 positives per query.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Querying FAISS index: 100%|██████████| 1/1 [00:00<00:00, 52.37it/s]


Metric       Positive       Negative     Difference
Count             976            974               
Mean           0.8118         0.5376         0.2751
Median         0.8327         0.5386         0.2795
Std            0.0933         0.0422         0.0735
Min            0.3503         0.2731         0.1053
25%            0.7721         0.5167         0.2257
50%            0.8327         0.5386         0.2800
75%            0.8755         0.5631         0.3292
Max            0.9637         0.6621         0.4420
Skipped 3,591 potential negatives (1.84%) due to the absolute_margin of 0.1.
Skipped 13 potential negatives (0.01%) due to the max_score of 0.8.
Could not find enough negatives for 2 samples (0.20%). Consider adjusting the range_max, range_min, absolute_margin, relative_margin and max_score parameters if you'd like to find more valid negatives.


(Dataset({
     features: ['question', 'context', 'negative'],
     num_rows: 974
 }),
 {'question': 'Trường hợp một SV được CBCT cho xem bài của SV khác trong giờ thi được giải quyết như thế nào?',
  'context': 'Điều 22. Xử lý cán bộ vi phạm quy định Trong thời gian thi, nếu các cán bộ thanh tra thi phát hiện thấy các trường hợp vi phạm thì lập biên bản tại chỗ và đề nghị Hiệu trưởng xử lý. Người tham gia công tác thi có hành vi vi phạm quy định (bị phát hiện trong khi làm nhiệm vụ hoặc sau kỳ thi), nếu có đủ chứng cứ, tùy theo mức độ, sẽ bị xử lý theo các hình thức sau đây. 1. Khiển trách đối với những người phạm lỗi nhẹ trong khi thi hành nhiệm vụ. 2. Cảnh cáo đối với những người vi phạm một trong các lỗi sau đây: a. CBCT bỏ coi thi mà không thông báo cho đơn vị tổ chức môn thi từ 2 lần trở lên trong một đợt thi. b. CBCT để cho SV tự do trao đổi khi làm bài, sử dụng tài liệu bị cấm sử dụng hoặc sử dụng các phương tiện thông tin liên lạc tại phòng thi. c. Cán bộ chấm thi chấm điểm ho

In [22]:
test_hard = make_hard_dataset(hf_test, num_negatives=1)
test_hard, test_hard[0]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Querying FAISS index: 100%|██████████| 1/1 [00:00<00:00, 53.16it/s]


Metric       Positive       Negative     Difference
Count             976            976               
Mean           0.8079         0.5385         0.2695
Median         0.8304         0.5399         0.2752
Std            0.0926         0.0462         0.0697
Min            0.3748         0.2659         0.1061
25%            0.7693         0.5150         0.2224
50%            0.8305         0.5400         0.2753
75%            0.8729         0.5675         0.3208
Max            0.9553         0.6447         0.4433
Skipped 3,811 potential negatives (1.94%) due to the absolute_margin of 0.1.
Skipped 6 potential negatives (0.00%) due to the max_score of 0.8.


(Dataset({
     features: ['question', 'context', 'negative'],
     num_rows: 976
 }),
 {'question': 'Trưởng Ban điều hành CTGT cần phê duyệt những cái gì?',
  'context': 'Điều 7. Quy trình tổ chức biên soạn giáo trình – Chủ biên xây dựng dự thảo đề cương chi tiết giáo trình và báo cáo trước Hội đồng khoa học và Đào tạo cấp Khoa. – Khoa thực hiện thủ tục đăng ký thực hiện giáo trình với Ban điều hành CTGT vào đầu tháng 11 hàng năm. – Trưởng Ban điều hành CTGT duyệt danh sách giáo trình đăng ký và ký hợp đồng biên soạn giáo trình với chủ biên theo đề cương đã được Khoa duyệt. – Chủ biên tạm ứng kinh phí tại Phòng Kế hoạch Tài chính và chi tạm ứng cho các cá nhân tham gia biên soạn. – Chủ biên tổ chức biên soạn bản thảo. – Trưởng Ban điều hành CTGT trường ra quyết định mời cán bộ phản biện. Chức danh khoa học, học vị của người phản biện không được thấp hơn chức danh khoa học, học 

In [23]:
# recheck hard dataset to make sure that 'context' columns and 'negative' column are not identical

def is_qualified(dataset):
    for idx, sample in enumerate(dataset):
        if sample['context'] == sample['negative']:
            return False
    return True
        
print(is_qualified(train_hard))
print(is_qualified(valid_hard))
print(is_qualified(test_hard))

True
True
True


In [117]:
# Initialize CrossEncoder model

model = CrossEncoder("KhoaUIT/Halong-UIT-R2GQA", num_labels=1, max_length=512, device='cuda') # num_labels=1 is for rerankers

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at KhoaUIT/Halong-UIT-R2GQA and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
# model_phobert = CrossEncoder("KhoaUIT/Phobert-UIT-R2GQA", num_labels=1, max_length=256, device='cuda')

# print(f"Halong tokenizer is_fast: {model.tokenizer.is_fast}") # True
# print(f"Phobert tokenizer is_fast: {model_phobert.tokenizer.is_fast}") # False

# for some reason, using "KhoaUIT/Phobert-UIT-R2GQA" as Cross-Encoder to evaluate dataset causing following warning:
# Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.how ever
# However, both models still work

In [26]:
from sentence_transformers.cross_encoder.evaluation import CrossEncoderRerankingEvaluator

def get_evaluator(dataset=None, name=None, k=10):
    assert dataset != None, "you forgot to pass your dataset"
    assert name != None, "you should set name of evaluator"

    samples = [
        {
            "query": sample["question"],
            "positive": [sample["context"]],   # ground truth relevant doc
            "negative": [sample["negative"]],  # hard negative
        }
        for sample in dataset  
    ]

    # Initialize the evaluator
    reranking_evaluator = CrossEncoderRerankingEvaluator(
        samples=samples,
        name=name,
        at_k=k,
        show_progress_bar=True,
        always_rerank_positives=True,  # since we’re using negatives explicitly
    )

    return reranking_evaluator

In [23]:
valid_evaluator = get_evaluator(valid_hard, "valid")
results = valid_evaluator(model)

# KhoaUIT/Phobert-UIT-R2GQA: {'valid_map': 0.7582135523613963, 'valid_mrr@10': 0.7582135523613963, 'valid_ndcg@10': 0.8215276323738772}
# KhoaUIT/Halong-UIT-R2GQA:  {'valid_map': 0.75564681724846,   'valid_mrr@10': 0.75564681724846,   'valid_ndcg@10': 0.8196330212525806}
print(results)

{'valid_map': 0.7453798767967146, 'valid_mrr@10': 0.7453798767967146, 'valid_ndcg@10': 0.8120545767673951}


In [44]:
# torch.cuda.empty_cache()

**Usage:**

[**BatchSamplers.NO_DUPLICATES**](https://sbert.net/docs/package_reference/sentence_transformer/sampler.html#sentence_transformers.sampler.NoDuplicatesBatchSampler): This sampler creates batches such that each batch contains samples where the values are unique, even across columns. This is useful when losses consider other samples in a batch to be in-batch negatives, and you want to ensure that the negatives are not duplicates of the anchor/positive sample.

In [121]:
args = CrossEncoderTrainingArguments(
    # Required parameter:
    output_dir="reranker-Halong",
    
    # Optional training parameters:
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    dataloader_drop_last=True,
    learning_rate=1e-5,
    warmup_ratio=0.1,
    fp16=True,   
    bf16=False,
    batch_sampler= BatchSamplers.NO_DUPLICATES, # without setting dataloader_drop_last=True, BatchSamplers.NO_DUPLICATES will raise error if there are any duplicates in a batch, it will remove them and lead to invalid batch. 
                                                # using default BatchSamplers.BATCH_SAMPLER works well but MultipleNegativesRankingLoss remmcomends to use BatchSamplers.NO_DUPLICATES which is better option
    
    # Optional tracking/debugging parameters:
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss', # we aim to minimize train loss, so using val loss to monitor is easily comparable
    greater_is_better=False,
    save_total_limit=1,
    eval_strategy="steps",
    eval_steps=250,
    save_strategy="steps", # option "best" does not work, 'best_model_checkpoint' in trainer state is None => use "epoch" instead if needed
    save_steps=250,
    logging_strategy="steps",
    logging_steps=250,
    logging_dir="logs",
    report_to="none"
)

# [IMPORTANT]:
# (1): num_negatives in MultipleNegativesRankingLoss (which differs from the one of Sentence Transformer) must match num_negatives in hard dataset obtained by fn mine_hard_negatives()
# (2): MultipleNegativesRankingLoss expects to work with option BatchSamplers.NO_DUPLICATES in CrossEncoderTrainingArguments
loss = losses.MultipleNegativesRankingLoss(model, num_negatives=1) 

early_stop = EarlyStoppingCallback(2)

trainer = CrossEncoderTrainer(
    model=model,
    args=args,
    train_dataset=train_hard,
    eval_dataset=valid_hard,
    loss=loss,
    callbacks=[early_stop]
)

In [122]:
train_loader = trainer.get_train_dataloader()
valid_loader = trainer.get_eval_dataloader()

len(train_loader), len(valid_loader), ('eval_step should be', 0.25*len(train_loader))

(975, 121, ('eval_step should be', 243.75))

In [75]:
# observe each batch after CrossEncoderTrainingArguments configuration

from collections import Counter

def debug_batches(dataloader):
    print('total batches:', len(dataloader))

    for i, batch in enumerate(dataloader):
        print(f"\n📦 Batch {i+1}")
        all_sample_values = []

        # Go through each sample in a single batch
        batch_size = len(batch['question'])
        for j in range(batch_size):
            sample_values = {
                str(batch[key][j]) for key in batch
            }
            all_sample_values.extend(sample_values)

        # Count and show duplicates
        counter = Counter(all_sample_values)
        num_duplicates = sum(1 for count in counter.values() if count > 1)

        for key, tensor in batch.items():
            print(f"   🔹 {key}: shape {len(tensor)}")
        print(f"   🔍 Duplicated values in this batch: {num_duplicates}")

In [77]:
# debug_batches(valid_loader)

In [123]:
print('------ model is traing now -----')

trainer.train()
trainer.save_model("./model_final")

------ model is traing now -----


Step,Training Loss,Validation Loss
250,0.935100,0.172875
500,0.121900,0.074566
750,0.090900,0.067739
1000,0.055800,0.060267
1250,0.045200,0.042803
1500,0.036400,0.043180
1750,0.046800,0.033441
2000,0.023800,0.051764
2250,0.022400,0.044976


In [124]:
import glob
import os

# Path to the folder
path = '/kaggle/working/reranker-Halong'

# Get all directories starting with 'checkpoint'
checkpoint_dirs = glob.glob(os.path.join(path, 'checkpoint*'))

if checkpoint_dirs != []:
    trainer_state_dir = os.path.join(checkpoint_dirs[0], 'trainer_state.json')
    print(trainer_state_dir)
    
else:
    print('checkpoint_dirs is empty:', checkpoint_dirs)

/kaggle/working/reranker-Halong/checkpoint-1750/trainer_state.json


In [125]:
if checkpoint_dirs != []:
    import json
    import pprint

    with open(trainer_state_dir, "r") as f:
        trainer_state = json.load(f)
        
    print('best checkpoint:', trainer.state.best_model_checkpoint, end='\n\n')
    pprint.pprint(dict(trainer_state))

best checkpoint: reranker-Halong/checkpoint-1750

{'best_global_step': 1750,
 'best_metric': 0.033440593630075455,
 'best_model_checkpoint': 'reranker-Halong/checkpoint-1750',
 'epoch': 1.7948717948717947,
 'eval_steps': 250,
 'global_step': 1750,
 'is_hyper_param_search': False,
 'is_local_process_zero': True,
 'is_world_process_zero': True,
 'log_history': [{'epoch': 0.2564102564102564,
                  'grad_norm': 7.725590705871582,
                  'learning_rate': 8.395904436860068e-06,
                  'loss': 0.9351,
                  'step': 250},
                 {'epoch': 0.2564102564102564,
                  'eval_loss': 0.17287543416023254,
                  'eval_runtime': 42.4157,
                  'eval_samples_per_second': 22.963,
                  'eval_steps_per_second': 2.876,
                  'step': 250},
                 {'epoch': 0.5128205128205128,
                  'grad_norm': 19.961320877075195,
                  'learning_rate': 9.232522796352584e-06,
 

In [126]:
model_best = CrossEncoder(trainer.state.best_model_checkpoint, num_labels=1, max_length=512, device='cuda')

In [ ]:
valid_evaluator = get_evaluator(valid_hard, "valid")
results = valid_evaluator(model_best)

print(results)

{'valid_map': 0.9969199178644764, 'valid_mrr@10': 0.9969199178644764, 'valid_ndcg@10': 0.9977264666544443}


In [ ]:
test_evaluator = get_evaluator(test_hard, "test")
results = test_evaluator(model_best)

print(results)

{'test_map': 0.9969262295081968, 'test_mrr@10': 0.9969262295081968, 'test_ndcg@10': 0.9977311255342508}


In [ ]:
model_best.push_to_hub('KhoaUIT/Halong-CrossEncoder-UIT-R2GQA')

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

'https://huggingface.co/KhoaUIT/KhoaUIT-CrossEncoder-UIT-R2GQA/commit/61bf54460d640cc5efd24e0aaa2fba014711d1cc'